In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
df.head()

In [ ]:
df.shape

Dataset contains 10127 data and 23 variables.

In [ ]:
#dropping the last two columns, we don't need them
df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'],axis=1,inplace=True)

#dividing the dataset into feature set and target set.
x=df.drop('Attrition_Flag',axis=1)
y=df['Attrition_Flag']

In [ ]:
#getting dummy variables for object type data
obj_list=x.select_dtypes(include='object')
float_list=x.select_dtypes(exclude='object')
obj_list=pd.get_dummies(obj_list)
x=pd.concat([obj_list,float_list],axis=1)

In [ ]:
# Existing Customer => 1
# Attrited Customer => 0
y.replace(['Existing Customer','Attrited Customer'],[1,0],inplace=True)
#I have simply replace "existing customer" and "Attrited Customer" of our target set by 1 and 0 respectively.
#One can use it using LaberEncoding

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.countplot(y)

In [ ]:
#The data is highly imbalanced. 
#proportion of Existing Customer and Attrited Customer to the dataset.
print(" Existing Customer : ",len(df[df['Attrition_Flag']=='Existing Customer'])/len(df))
print(" Attrited Customer : ",len(df[df['Attrition_Flag']=='Attrited Customer'])/len(df))

83% of the data contains Data of Existing Customer and only 17% data contains data of Attrited Customer.<br>If we use this data to train any model than the model would be biased towards Existing Customer.<br>To deal with this we would need to use some under sampling techniques or over sampling techniques to balance the data.<br>Here I am using SMOTE to deal with this.

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(random_state=60)
x_smote,y_smote=smote.fit_resample(x,y)

In [ ]:
sns.countplot(y_smote)

After using SMOTE,we can see the data is now balanced. so, we could proceed towards training the model.

In [ ]:
#splitting given dataset into training set and testing set using train_test_split. 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_smote,y_smote,test_size=0.2,random_state=4)
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
rf=make_pipeline(StandardScaler(),RandomForestClassifier(n_estimators=1000))

In [ ]:
rf.fit(x_train,y_train)
yhat=rf.predict(x_test)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresh = roc_curve(y_test,rf.predict_proba(x_test)[:,1], pos_label=1)
plt.plot(fpr, tpr,label='Random Forest Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

The ROC curve shows the trade-off between sensitivity (or TPR) and specificity (1 – FPR). Classifiers that give curves closer to the top-left corner indicate a better performance.Here we can see that the curve is very close to top-left corner, which shows the performance of our model is good in our dataset.

In [ ]:
from sklearn.metrics import roc_auc_score,classification_report
print(classification_report(yhat,y_test))
print("roc auc score : ",roc_auc_score(yhat,y_test))

The model classifies the data with 97% precision,recall and roc_auc_score.

Let me know how could i improve this model.<br>
Thank You.